# 형태소 분석
- 텍스트 데이터에서 단어를 이루는 가장 작은 의미 단위인 형태소(Morpheme)를 분석하고 추출하는 과정
- 자연어 처리(NLP)의 기초 단계로, 문장을 구성하는 단어를 분해하고, 각각의 단어가 가지는 의미적, 문법적 정보를 분석하는 데 사용
- 형태소를 비롯하여, 어근, 접두사/접미사, 품사 등 다양한 언어적 속성의 구조를 파악



## 형태소란?
- 의미를 가지는 가장 작은 언어 단위
- 예:
    - 한국어: "사람들" → "사람" (명사) + "들" (복수 접미사)
    - 영어: "unhappy" → "un" (부정 접두사) + "happy" (형용사)
- 형태소는 크게 두가지로 나뉜다.
    - 자립형태소: 혼자서도 의미를 가지는 형태소
    > 예: 명사, 동사, 형용사 등
    - 의존형태소: 단독으로는 의미를 가지지 못하고 다른 형태소와 결합하여 의미를 가지는 형태소
    > 예: 조사, 접사, 어미 등

## 언어별 특징
- 한국어
    - 조사가 붙는 교착어로, 한 단어에 여러 형태소가 결합
    > 예: "사람들이" → "사람" (명사) + "들" (복수) + "이" (주격 조사)
    - 조사, 어미 등이 문법적으로 중요한 역할을 하므로, 형태소 분석이 NLP에 필수
- 영어
    - 분석이 비교적 단순하며, 주로 접사(un-, -ed)나 동사 변화 분석에 집중
    > 예: "running" → "run" (어근) + "-ing" (현재분사 접미사)

## 형태소 분석기
- 형태소 분석기는 품사를 태깅해주는 라이브러리  
- 영어에서의 품사는 문장에서 단어들의 위치가 띄어쓰기 단위로 되어 있기 때문에 POS(Part of Speech) tagger라고 함
- 반면에 한국어에서는 단어를 다 잘라내야 제대로 형태소를 갈라낼 수 있어서 Morphology Analyzer(형태학적 분석)라고 함
  


# 뉴스 분류 데이터셋
- 학습 데이터
    - https://drive.google.com/file/d/1-DFxEF9otbqt-swnM1fXVWAFIweiR1qM/view?usp=sharing
- 테스트 데이터
    - https://drive.google.com/file/d/1rL-LkmmM46V2HmLI1CKxK8LbdpP2pzvT/view?usp=sharing

- 0 : 세계, 1: 스포츠. 2: 비즈니스, 3: 과학기술

In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "../data/"
SEED = 42
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# nltk
- python에서 가장 오래되고 유명한 자연어 처리 라이브러리
- 영어 품사 정보
    - https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [3]:
import nltk
nltk.download('punkt_tab') # 토크나이저 모델
nltk.download('stopwords') # 불용어 리스트
nltk.download('averaged_perceptron_tagger_eng') # 품사정보

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\kwon3\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kwon3\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\kwon3\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.


True

In [4]:
train = pd.read_csv(f"{DATA_PATH}train_news.csv")
test = pd.read_csv(f"{DATA_PATH}test_news.csv")

train.shape, test.shape

((89320, 3), (38280, 2))

In [5]:
train.head()

,title,desc,target
0,Sudan Postpones Decision to Expel Oxfam and Sa...,Sudan has decided to postpone a decision to ex...,0
1,Coming Soon: Mobile TV,Cell phone manufacturers are teaming up to bri...,2
2,Experts warn of Internet flu vaccine scam,Although the United States is experiencing a s...,3
3,Bollor ups Havas stake to 20.2,Corporate raider Vincent Bollor said yesterday...,2
4,"Hurricane Ivan Kills 20 in Grenada, Heads West...",Reuters - Hurricane Ivan killed at least 20 pe...,0


In [6]:
test.head()

,title,desc
0,Mass. launches insurance probes,Massachusetts Attorney General Thomas F. Reill...
1,Jackson the Wizard of Loz,WHATEVER her status as an individual in the wo...
2,Coffee-Based Log Burns Cleaner -- But No Starb...,"Take an entrepreneur, add an interesting fact ..."
3,Annual Cell Phone Guide,Fast Forward columnist Rob Pegoraro was online...
4,Casino workers end strike in Atlantic City,"ATLANTIC CITY, NJ -- Thousands of cocktail wai..."


In [7]:
text = train["desc"].loc[0]
text

'Sudan has decided to postpone a decision to expel the heads of two British aid agencies - Oxfam and Save the Children - citing administrative difficulties and humanitarian grounds.'

## 토큰화

In [8]:
from nltk.tokenize import word_tokenize

word_tokenize(text)

['Sudan',
 'has',
 'decided',
 'to',
 'postpone',
 'a',
 'decision',
 'to',
 'expel',
 'the',
 'heads',
 'of',
 'two',
 'British',
 'aid',
 'agencies',
 '-',
 'Oxfam',
 'and',
 'Save',
 'the',
 'Children',
 '-',
 'citing',
 'administrative',
 'difficulties',
 'and',
 'humanitarian',
 'grounds',
 '.']

## 불용어

In [9]:
from nltk.corpus import stopwords

stopwords.words("english")[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [10]:
len(stopwords.words("english"))

179

## 품사 태깅

In [11]:
tokens = word_tokenize(text)
nltk.tag.pos_tag(tokens)

[('Sudan', 'NNP'),
 ('has', 'VBZ'),
 ('decided', 'VBN'),
 ('to', 'TO'),
 ('postpone', 'VB'),
 ('a', 'DT'),
 ('decision', 'NN'),
 ('to', 'TO'),
 ('expel', 'VB'),
 ('the', 'DT'),
 ('heads', 'NNS'),
 ('of', 'IN'),
 ('two', 'CD'),
 ('British', 'JJ'),
 ('aid', 'NN'),
 ('agencies', 'NNS'),
 ('-', ':'),
 ('Oxfam', 'NNP'),
 ('and', 'CC'),
 ('Save', 'NNP'),
 ('the', 'DT'),
 ('Children', 'NNP'),
 ('-', ':'),
 ('citing', 'VBG'),
 ('administrative', 'JJ'),
 ('difficulties', 'NNS'),
 ('and', 'CC'),
 ('humanitarian', 'JJ'),
 ('grounds', 'NNS'),
 ('.', '.')]

- N or V or J 로 시작하는 품사들의 단어만 다시 새로운 리스트로 담기

In [12]:
result = [t for t, pos in nltk.tag.pos_tag(tokens) if pos.startswith(("V", "N", "J"))]
result

['Sudan',
 'has',
 'decided',
 'postpone',
 'decision',
 'expel',
 'heads',
 'British',
 'aid',
 'agencies',
 'Oxfam',
 'Save',
 'Children',
 'citing',
 'administrative',
 'difficulties',
 'humanitarian',
 'grounds']

In [13]:
train["clean"] = train["desc"].str.replace("[^\w ]+", "", regex=True).str.lower()
test["clean"] = test["desc"].str.replace("[^\w ]+", "", regex=True).str.lower()

## 표제어

In [14]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kwon3\AppData\Roaming\nltk_data...


True

In [15]:
from nltk.stem import WordNetLemmatizer

In [16]:
wnl = WordNetLemmatizer()
wnl.lemmatize("dogs")

'dog'

In [17]:
wnl.lemmatize("is")

'is'

In [18]:
wnl.lemmatize("has")

'ha'

- "v"(동사), "a"(형용사), "n"(명사)

In [19]:
wnl.lemmatize("is", "v")

'be'

In [20]:
wnl.lemmatize("has", "v")

'have'

# spaCy
- 딥러닝 기반의 형태소 분석 라이브러리

In [22]:
import spacy

In [25]:
nlp = spacy.load("en_core_web_sm")
nlp

In [26]:
text = train["clean"].iloc[0]
text

'sudan has decided to postpone a decision to expel the heads of two british aid agencies  oxfam and save the children  citing administrative difficulties and humanitarian grounds'

In [27]:
doc = nlp(text)
type(doc) # Doc 객체

spacy.tokens.doc.Doc

In [28]:
len(doc) # 29개의 토큰 객체

29

In [29]:
doc[1] # 인덱싱 가능

has

In [30]:
type(doc[1]) # 토큰 객체

spacy.tokens.token.Token

In [31]:
doc[1].text # 원래 단어

'has'

In [32]:
doc[1].lemma_ # 표제어

'have'

In [33]:
doc[1].tag_ # 품사

'VBZ'

In [34]:
doc[1].is_alpha # 알파벳 여부

True

In [35]:
doc[1].is_stop # 불용어 여부

True

In [36]:
cols = ["단어", "표제어", "품사", "알파벳여부", "불용어여부"]
data = [(token.text, token.lemma_, token.tag_, token.is_alpha, token.is_stop) for token in doc]
df = pd.DataFrame(data, columns=cols)
df

,단어,표제어,품사,알파벳여부,불용어여부
0,sudan,sudan,NNP,True,False
1,has,have,VBZ,True,True
2,decided,decide,VBN,True,False
3,to,to,TO,True,True
4,postpone,postpone,VB,True,False
5,a,a,DT,True,True
6,decision,decision,NN,True,False
7,to,to,TO,True,True
8,expel,expel,VB,True,False
9,the,the,DT,True,True


- tokenizer 메서드 사용 시 속도는 빠르지만, 품사/표제어 정보 추출 x

In [37]:
doc = nlp.tokenizer(text)
doc

sudan has decided to postpone a decision to expel the heads of two british aid agencies  oxfam and save the children  citing administrative difficulties and humanitarian grounds

In [38]:
doc[1].lemma_

''

In [39]:
doc[1].tag_

''

- 품사가 N, V, J, R로 시작하는 토큰들만 토큰화

In [ ]:
# train_list = []

# for text in tqdm(train["clean"]):
#     doc = nlp(text)
#     tmp = [t for t in doc if t.tag_[0] in "NVJR"]
#     train_list.append(tmp)

In [40]:
train_list = []

for text in tqdm(train["clean"]):
    doc = nlp.tokenizer(text)
    tmp = [t for t in doc if not t.is_alpha]
    train_list.append(tmp)

  0%|          | 0/89320 [00:00<?, ?it/s]

- nltk 활용해서 불용어 제거와 명사, 동사, 형용사, 부사만 토큰화해서 train_list에 담기
    - test_list에도 동일하게 작업

In [41]:
train_list = []
stop_words = stopwords.words("english")

for text in tqdm(train["clean"]):
    token = word_tokenize(text)
    words = [t for t, pos in nltk.pos_tag(token) if t not in stop_words and pos[0] in ("NVJR")]
    train_list.append(" ".join(words))

  0%|          | 0/89320 [00:00<?, ?it/s]

In [42]:
test_list = []

for text in tqdm(test["clean"]):
    token = word_tokenize(text)
    words = [t for t, pos in nltk.pos_tag(token) if t not in stop_words and pos[0] in ("NVJR")]
    test_list.append(" ".join(words))

  0%|          | 0/38280 [00:00<?, ?it/s]

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(max_features=500)
train_data = vec.fit_transform(train_list).toarray()
test_data = vec.transform(test_list).toarray()

In [44]:
train_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [45]:
test_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)